In [30]:
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime

In [31]:
# Function to get options data for a given ticker
def get_options_data(ticker):
    stock = yf.Ticker(ticker)
    expirations = stock.options
    options_data = {}

    for expiration_date in expirations:
        options_chain = stock.option_chain(expiration_date)
        options_data[expiration_date] = {
            'calls': options_chain.calls.to_dict(orient='records'),
            'puts': options_chain.puts.to_dict(orient='records')
        }

    return options_data

# Example ticker
ticker = 'SPY'
options_data = get_options_data(ticker)
print(options_data)

# Get historical prices
history = yf.Ticker(ticker).history(period='1d')
last_price = history['Close'].iloc[-1]

# Convert expiration dates to datetime objects
expiration_dates = [datetime.strptime(date, '%Y-%m-%d') for date in options_data.keys()]

# Create an empty list to store traces
traces = []

{'2024-01-03': {'calls': [{'contractSymbol': 'SPY240103C00402000', 'lastTradeDate': Timestamp('2024-01-02 14:57:32'), 'strike': 402.0, 'lastPrice': 67.3, 'bid': 67.45, 'ask': 67.65, 'change': -2.3499985, 'percentChange': -3.3740106, 'volume': 1.0, 'openInterest': 6, 'impliedVolatility': 1.1484417578125, 'inTheMoney': True, 'contractSize': 'REGULAR', 'currency': 'USD'}, {'contractSymbol': 'SPY240103C00404000', 'lastTradeDate': Timestamp('2024-01-03 15:01:30'), 'strike': 404.0, 'lastPrice': 66.09, 'bid': 65.54, 'ask': 65.75, 'change': -5.6100006, 'percentChange': -7.824269, 'volume': 1.0, 'openInterest': 5, 'impliedVolatility': 1.29297228515625, 'inTheMoney': True, 'contractSize': 'REGULAR', 'currency': 'USD'}, {'contractSymbol': 'SPY240103C00405000', 'lastTradeDate': Timestamp('2023-12-29 19:49:43'), 'strike': 405.0, 'lastPrice': 70.69, 'bid': 64.21, 'ask': 64.82, 'change': 0.0, 'percentChange': 0.0, 'volume': 10.0, 'openInterest': 6, 'impliedVolatility': 0.9062509375, 'inTheMoney': Tru

In [32]:
# Loop through each expiration date
for expiration_date in expiration_dates:
    formatted_date = expiration_date.strftime('%Y-%m-%d')
    
    # Extract data for visualization
    strike_prices_calls = [float(call['strike']) for call in options_data[formatted_date]['calls']]
    strike_prices_puts = [float(put['strike']) for put in options_data[formatted_date]['puts']]
    iv_calls = [float(call['impliedVolatility']) for call in options_data[formatted_date]['calls']]
    iv_puts = [float(put['impliedVolatility']) for put in options_data[formatted_date]['puts']]
    
    # Get day of the year for the expiration date
    expiration_day_of_year = expiration_date.timetuple().tm_yday
    
    # Create traces for call and put options
    trace_calls = go.Scatter3d(
        x=strike_prices_calls,
        y=[expiration_day_of_year] * len(strike_prices_calls),
        z=iv_calls,
        mode='markers',
        marker=dict(size=5, color='blue'),  # Set color for call options (e.g., blue)
        name=f'Call Options - {formatted_date}'
    )
    
    trace_puts = go.Scatter3d(
        x=strike_prices_puts,
        y=[expiration_day_of_year] * len(strike_prices_puts),
        z=iv_puts,
        mode='markers',
        marker=dict(size=5, color='red'),  # Set color for put options (e.g., red)
        name=f'Put Options - {formatted_date}'
    )
    
    # Append traces to the list
    traces.extend([trace_calls, trace_puts])

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='Strike Price'),
        yaxis=dict(title='Expiration Day of Year'),
        zaxis=dict(title='Implied Volatility'),
    ),
    title=f'Volatility Surface for {ticker} - Last Price: {last_price}',
    showlegend=True
)

In [33]:
# Create figure
fig = go.Figure(data=traces, layout=layout)

# Save the HTML file
html_file_path = 'volatility_surface.html'
fig.write_html(html_file_path)

# Print the path to the HTML file
print(f'Volatility surface saved to: {html_file_path}')

Volatility surface saved to: volatility_surface.html
